In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
import numpy as np

import Audio_functions as af
from functools import partial
import datetime

import librosa
import soundfile as sf
import IPython.display as ipd

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
data_folder = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/backgrounds'
#data_folder = 'C:/Users/Toby/Speech_Data/LibriSpeech/train-other-500'
sample_rate = 16384
n_parallel_readers = 4
n_channels = 1

In [7]:
data_pipe = tf.data.Dataset.list_files(data_folder + '/*.wav').map(partial(af.read_audio,
                                                                            sample_rate=sample_rate,
                                                                            n_channels=n_channels), 
                                                                    num_parallel_calls=n_parallel_readers)

In [8]:
sess = tf.Session()
next_file = data_pipe.make_one_shot_iterator().get_next()

In [9]:
length = 0
file_count = 0
max_file_length = 0
min_file_length = 100000

while True:
    try:
        file = sess.run(next_file)
        len_file = len(file) / sample_rate
        length += len_file
        if len_file > max_file_length:
            max_file_length = len_file
        if len_file < min_file_length:
            min_file_length = len_file
        file_count += 1
        if file_count % 10 == 0:
            print('{fc} files processed'.format(fc=file_count))
    except tf.errors.OutOfRangeError:
        mean_file_length = length / file_count
        print('Files in folder: {fc}'.format(fc=file_count))
        print('Total length of audio in folder = {l}s ({lh} hours)'.format(l=round(length, 2), 
                                                                           lh=str(datetime.timedelta(seconds=length))))
        print('Maximum file length = {maxfl}s'.format(maxfl=round(max_file_length, 2)))
        print('Minimum file length = {minfl}s'.format(minfl=round(min_file_length, 2)))
        print('Mean file length = {meanfl}s'.format(meanfl=round(mean_file_length, 2)))
        break

10 files processed
20 files processed
30 files processed
40 files processed
50 files processed
60 files processed
70 files processed
80 files processed
90 files processed
100 files processed
Files in folder: 102
Total length of audio in folder = 181894.61s (2 days, 2:31:34.611328 hours)
Maximum file length = 2865.91s
Minimum file length = 1008.62s
Mean file length = 1783.28s
